# Create CNN Model and Optimize it Using Keras TUner

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.5.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                                kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
                                activation='relu',
                                input_shape=(28,28,1)
                            ),
                             keras.layers.Conv2D(
                                filters = hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                                kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
                                activation='relu',
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units = hp.Int('dense_1_units',min_value=32, max_value=128, step=16),
                                activation='relu'
                            ),
                            keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
             loss ='sparse_categorical_crossentropy',
             metrics = ['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='Fashion Mnist')

In [ ]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 50m 22s]
val_accuracy: 0.8733333349227905

Best val_accuracy So Far: 0.9104999899864197
Total elapsed time: 02h 23m 21s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 96)        41568     
_________________________________________________________________
flatten (Flatten)            (None, 46464)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2973760   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 3,017,226
Trainable params: 3,017,226
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1990 - accuracy: 0.9256 - val_loss: 0.2519 - val_accuracy: 0.9107
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1547 - accuracy: 0.9424 - val_loss: 0.2480 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1165 - accuracy: 0.9564 - val_loss: 0.2675 - val_accuracy: 0.9105
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0872 - accuracy: 0.9672 - val_loss: 0.3237 - val_accuracy: 0.9105
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0660 - accuracy: 0.9751 - val_loss: 0.3397 - val_accuracy: 0.9128
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0507 - accuracy: 0.9814 - val_loss: 0.3944 - val_accuracy: 0.9110
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0412 - accuracy: 0.9851 - val_loss: 0.4168 - val_a